# Train Drone Detection Model on Google Colab (FREE GPU)

**Setup Instructions:**
1. Upload this notebook to Google Colab
2. Go to Runtime → Change runtime type → Select **T4 GPU**
3. Run all cells

Training time: ~5-10 minutes on GPU (vs hours on Mac CPU)

## 1. Install Dependencies

In [ ]:
!pip install ultralytics roboflow -q

## 2. Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow

# Initialize Roboflow with your API key
API_KEY = "aO6VaU58uJ4WMpjdwVWU"  # Replace with your key if different
rf = Roboflow(api_key=API_KEY)

# Download drone-vs-bird dataset
project = rf.workspace("oleksandr-gorpynich").project("drone-vs-bird-object-detection-2fnnk")
dataset = project.version(1).download("yolov8")

print(f"Dataset downloaded to: {dataset.location}")

## 3. Convert to Binary Dataset (drone vs not-drone)

In [ ]:
import yaml
import shutil
from pathlib import Path

def create_binary_dataset(dataset_path, output_path="drone_binary"):
    dataset_path = Path(dataset_path)
    output_path = Path(output_path)
    
    # Read data.yaml
    with open(dataset_path / "data.yaml", 'r') as f:
        data_config = yaml.safe_load(f)
    
    class_names = data_config['names']
    drone_idx = class_names.index('drone')
    
    print(f"Original classes: {class_names}")
    print(f"Converting to binary: drone (0) vs not-drone (1)")
    
    # Process each split
    for split in ['train', 'valid', 'test']:
        split_path = dataset_path / split
        if not split_path.exists():
            continue
        
        out_images = output_path / split / "images"
        out_labels = output_path / split / "labels"
        out_images.mkdir(parents=True, exist_ok=True)
        out_labels.mkdir(parents=True, exist_ok=True)
        
        images_path = split_path / "images"
        labels_path = split_path / "labels"
        
        count = 0
        for label_file in labels_path.glob("*.txt"):
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            # Relabel: drone=0, everything else=1
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                class_id = int(parts[0])
                parts[0] = '0' if class_id == drone_idx else '1'
                new_lines.append(' '.join(parts) + '\n')
            
            # Copy image
            image_name = label_file.stem
            for ext in ['.jpg', '.jpeg', '.png']:
                image_file = images_path / f"{image_name}{ext}"
                if image_file.exists():
                    shutil.copy2(image_file, out_images / f"{image_name}{ext}")
                    break
            
            # Write relabeled annotations
            with open(out_labels / label_file.name, 'w') as f:
                f.writelines(new_lines)
            count += 1
        
        print(f"{split}: {count} images")
    
    # Create new data.yaml
    new_data_config = {
        'path': str(output_path.absolute()),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'names': {0: 'drone', 1: 'not-drone'},
        'nc': 2
    }
    
    with open(output_path / "data.yaml", 'w') as f:
        yaml.dump(new_data_config, f)
    
    print(f"\n✓ Binary dataset created at: {output_path}")
    return output_path

# Convert dataset
binary_dataset = create_binary_dataset(dataset.location)
data_yaml = binary_dataset / "data.yaml"

## 4. Train YOLOv8 Model (Fast on GPU!)

In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')

# Train
results = model.train(
    data=str(data_yaml),
    epochs=25,
    imgsz=640,
    batch=16,
    name='drone_detection',
    device=0  # Use GPU
)

print("\n✓ Training complete!")

## 5. Evaluate Model

In [ ]:
# Validate on test set
metrics = model.val()

print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

## 6. Test Inference

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Get test images
test_images = list(Path(binary_dataset / "test/images").glob("*.jpg"))[:6]

# Run inference
results = model.predict(test_images, conf=0.25)

# Display results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (result, ax) in enumerate(zip(results, axes)):
    img = result.plot()
    img = img[:, :, ::-1]  # BGR to RGB
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"{len(result.boxes)} detection(s)")

plt.tight_layout()
plt.show()

## 7. Download Trained Model

In [ ]:
# Find best model
import os
from google.colab import files

best_model = "runs/detect/drone_detection/weights/best.pt"

if os.path.exists(best_model):
    print("Downloading trained model...")
    files.download(best_model)
    print("✓ Download complete! Upload this to your Mac.")
else:
    print(f"Model not found at {best_model}")